<a href="https://colab.research.google.com/github/parkjeongmi/OptimalLocation/blob/master/%EA%B3%A0%EC%96%91%EC%8B%9C_%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0_%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%85%98_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고양시 공공자전거 스테이션 최적화

## 개요
 - 과제의 목표는 지역별 특성에 맞춰 일부 지역은 효율성 증대, 일부지역은 접근성 증대를 목적으로 스테이션 신규 설치 및 위치 재조정이 목표이다.
 - 그러기 위해 인구분포 및 유동인구를 나타내는 버스이용객, 지하철이용객 수와 각 건물의 분포, 공원의 유무 등의 요인을 통해 일일스테이션 평균을 예측하는 모델을 설계하고, 그 모델을 바탕으로 좀 더 효율적인 위치로 스테이션을 배치하고자 한다.

 

## 방법
- 공간정보 및 지하철, 버스 정보등을 이용하여 학습용 데이터를 생성한다.
- 일일평균 이용량 예측 모델 학습
- 후보 좌표들의 모델 데이터 생성
- 후보 좌표들의 일일평균 이용량 예측
- 예측한 데이터를 군집화하여 새로운 후보 스테이션 위치 생성
- 새로운 후보 스테이션과 기존 스테이션 간 비교를 통해 기존 스테이션 위치 조정 및 신규 스테이션 배치
- 새로운 스테이션 위치를 가장 가까운 도로로 배치 

# 1. 데이터 전처리

In [ ]:
!pip install geopandas
!pip install haversine
!pip install catboost

Traceback (most recent call last):



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import geopandas as gpd
from haversine import haversine
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from shapely.geometry import Point
import tensorflow as tf
from sklearn.cluster import KMeans

In [ ]:
oper_01 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/01.운영이력.csv")
buliding_point = gpd.read_file("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/15.도로명주소_건물.geojson")
station_2 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/02.자전거스테이션.csv")
spread_pop = gpd.read_file('/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/06.인구(거주)분포도(100M X 100M).geojson')
subway_19 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/19.전철역_공간정보.csv")
bus_20 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/20.고양시 버스정류소.csv")
useBus_21 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/21.버스 정류장별 승하차 정보.csv")
useSubway_29 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/29.지하철 역별 이용객수.csv") 
dong_point = gpd.read_file("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/34.행정경계(행정동기준).geojson")
도로명주소_도로 = gpd.read_file('/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/16.도로명주소_도로.geojson')
공간시설 = gpd.read_file('/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/10.도시계획(공간시설).geojson')

KeyboardInterrupt: ignored

### 스테이션별 300m 기준으로 건물개수 구하기
 - BDTYP_CD로 어떤 건물인지 파악하고 건물데이터에 있는 geometry값의 centroid와 스테이션 좌표간의 거리가 300m 이내인 건물들의 개수를 구하여 요소로 사용

#### 도로명 주소 데이터 확인

In [ ]:
buliding_point

In [ ]:
# 함수로 만들어서 모듈화 
# 시간이 오래걸림
def building_count(df) :
    # building_point에서 필요한 column만 추출
    buliding_point2 = buliding_point.copy()
    buliding_point2 = buliding_point2[['BDTYP_CD','geometry']]
    buliding_point2.rename(columns={'BDTYP_CD':'건물종류'},inplace=True)
    buliding_point2['centroid'] = buliding_point2.geometry.centroid

    # 스테이션과 building centroid와 비교해서 300m 이내 건물 추출
    station_list = []

    for i in range(len(df)):
        for j in range(len(buliding_point2)):
          x = (df.iloc[i].경도,df.iloc[i].위도)
          y = (buliding_point2.iloc[j].centroid.x,buliding_point2.iloc[j].centroid.y)
          result = haversine(x,y,unit='m')
          if result <= 300 :
           station_list.append([df.iloc[i].Station_ID,buliding_point2.iloc[j].건물종류,1])
    
    station_list = pd.DataFrame(station_list)
    station_list.rename(columns={0:'Station_ID',1:'건물종류',2:'건물수'},inplace=True)
    buliding_sum = station_list.groupby(['Station_ID','건물종류']).sum()
    buliding_sum[:] = 1 # building 데이터 같은 경우 있으면 1 없으면 0으로 카테고리컬하게 변수변경
    buliding_sum = pd.pivot_table(buliding_sum,
                              index='Station_ID',
                              columns='건물종류',
                              values='건물수')
    buliding_sum = buliding_sum.fillna(0)

    return buliding_sum

## 스테이션 주변 인구 구하기
- 인구분포도를 이용하여 스테이션 주변 300m 이내를 구함

#### 인구분포도 데이터 확인

In [ ]:
spread_pop

#### nan 값 0으로 변환

In [ ]:
spread_pop.loc[spread_pop['val'].apply(pd.isna), 'val'] = 0
spread_pop

#### 계산에 필요한 거리와 축적 구해두기

In [ ]:
거리 = 300
축척 = sum([
    *(spread_pop['geometry'].bounds['maxx'] - spread_pop['geometry'].bounds['minx']),
    *(spread_pop['geometry'].bounds['maxy'] - spread_pop['geometry'].bounds['miny']),
]) / (2 * len(spread_pop)) / 100
축척

In [ ]:
#만들고 싶은 좌표의 경도와 위도, 경도와 위도가 포함된 df , merge하고싶은 함수 넣기
def pop_count(경도, 위도, df1) :
    batch_size = 32

    인구분포도_경위도 = np.stack([spread_pop['geometry'].centroid.x, spread_pop['geometry'].centroid.y], axis=1).reshape([1, -1, 2])

    데이터 = pd.DataFrame()
    for i in tqdm(range(int(np.ceil(len(경도) / batch_size)))):
        s, e = batch_size * i, batch_size * (i + 1)
        자전거스테이션_경위도 = np.stack([경도[s:e], 위도[s:e]], axis=1).reshape([-1, 1, 2])
        인구분포_조건 = np.sum((자전거스테이션_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리 * 축척
        인구분포 = np.array([spread_pop.loc[x, 'val'].mean() for x in 인구분포_조건])
        
        
        데이터 = 데이터.append(pd.DataFrame(
            columns=['인구분포'],
            data=np.stack([인구분포], axis=1),
        ), ignore_index=True)
        
        merge_data = pd.merge(데이터, df1, how='left',left_index=True,right_index=True)
        merge_data.set_index('Station_ID',inplace=True)
    return merge_data

## 버스와 지하철 이용객 구하기
- 이것도 마찬가지로 station 좌표와 주변에 있는 버스정류장, 지하철의 좌표를 이용하여 300m 근방에 있는 이용객의 합을 구한다.

In [ ]:
# 버스 위치데이터와 버스 승차 데이터, 지하철데이터와 지하철 승차 데이터를 병합해준다.
# 먼저 버스데이터부터 병합을 시작한다.
# 버스데이터는 이름이 겹치는 경우가 있기 때문에 ID값을 통해 합쳐준다.
# 마찬가지로 함수로 만들어서 모듈화해준다.
def bus_count(df,df2) :
    newbus_data = pd.merge(bus_20,useBus_21,on=('STATION_ID','STATION_NM'))
    
    buslist = []

    for i in range(len(df)) :
        usesum = 0
        for j in range(len(newbus_data)) :

         x = (df.iloc[i].위도,df.iloc[i].경도)
         y = (newbus_data.iloc[j].Y,newbus_data.iloc[j].X)
         result1 = haversine(x,y,unit='m')
         if result1 <= 300 :
            usesum = usesum + newbus_data.iloc[j].GETON_CNT

        buslist.append([df.iloc[i].Station_ID,usesum])
        
    df_buslist = pd.DataFrame(buslist)
    df_buslist.rename(columns = {0 : 'Station_ID', 1 : '버스이용객'}, inplace = True)
    merge_data = pd.merge(df2,df_buslist,on='Station_ID')
    merge_data['버스이용객'] = merge_data['버스이용객']/365

    return merge_data

In [ ]:
# 지하철 함수
# 우리가 필요한건 역별 연간 총 승하차 인원으로 아래와 같이 데이터를 만들고 기존 지하철 위치데이터와 병합한다.
def sub_count(df,df2) :
    # 다음 지하철역 데이터를 합쳐준다. 먼저 각 데이터의 상태를 파악한다.
    # 확인 후 도로명주소 및 동 주소는 사용하지 않으므로 삭제한다.
    newsub_data = subway_19.drop(['lot_num_addr','road_nm_addr'],axis=1)
    new_useSub = useSubway_29[['역명','승하차구분','모든요일_합계']]
    new_useSub = new_useSub[new_useSub['승하차구분'] == '총승하차']
    #불필요하거나 겹치는 column들은 삭제해주고 이름도 변경해준다.
    newsub_data = pd.merge(newsub_data,new_useSub,left_on = 'station_nm', right_on='역명')
    newsub_data = newsub_data.drop(['승하차구분', '역명'] , axis = 1)
    newsub_data.rename(columns={"모든요일_합계":"use"},inplace=True)

    sublist = []

    for i in range(len(df)):
        usesum = 0
        for j in range(len(newsub_data)):
          x = (df.iloc[i].위도,df.iloc[i].경도)
          y = (newsub_data.iloc[j].Y,newsub_data.iloc[j].X)
          result1 = haversine(x,y,unit='m')
          if result1 <= 300 :
            usesum = usesum + newsub_data.iloc[j].use
        sublist.append([df.iloc[i].Station_ID,usesum])

    df_sublist = pd.DataFrame(sublist)
    df_sublist.rename(columns={0:"Station_ID",1:"지하철이용객"},inplace=True)

    merge_data = pd.merge(df2,df_sublist,on='Station_ID')
    
    merge_data['지하철이용객'] = merge_data['지하철이용객']/365
    #데이터 전체적으로 nan 값 0으로 바꿔주기
    merge_data = merge_data.fillna(0)

    return merge_data

## 스테이션 300m 이내 공원 구하기

In [ ]:
# 어떤 관리번호가 공원인지 파악하기 위해 데이터를 넣어서 파악
좌표_dic = {'name': ['백석근린공원@','알미공원@@@', '안산근린공원@', '일산문화공원@', '강촌근린공원옆', '그냥지하철출구'], 
          '위도':[37.645210, 37.646015, 37.646388, 37.658306, 37.653520, 37.652188], 
          '경도' : [126.795244, 126.783182, 126.784461,126.770618, 126.781920,126.778329]}
좌표 = pd.DataFrame.from_dict(좌표_dic)
좌표

- 해당 좌표를 가진 데이터들은 어떤 관리번호를 나타내는지 확인
- 공원의 시설관리번호는 UQT22를 포함하는 것을 알 수 있음

In [ ]:
for i in range(len(좌표)):
  for j in range(len(공간시설)):
    if Point(좌표.iloc[i].경도, 좌표.iloc[i].위도).within(공간시설.iloc[j].geometry) :
       print(좌표['name'].iloc[i], 공간시설['MNUM'].iloc[j])
       break
    continue

- UQT22를 활용하여 공원 데이터프레임 생성
- 총 107개의 공원 데이터 생성

In [ ]:
공간시설_UQT22 = 공간시설['MNUM'].str.contains('UQT22')
park_data = 공간시설[공간시설_UQT22]
park_data

In [ ]:
def park_count(df1,df2) :
    
    park_list = []

    for i in range(len(df1)):
        for j in range(len(park_data)):
          x = (df1.iloc[i].경도,df1.iloc[i].위도)
          y = (park_data.iloc[j].경도,park_data.iloc[j].위도)
          result = haversine(x,y,unit='m')
          if result <= 300 :
           park_list.append([df1.iloc[i].Station_ID,1])

    park_list = pd.DataFrame(park_list)
    park_list.rename(columns={0:'Station_ID',1:'공원수'},inplace=True)

    temp_data = park_list.groupby('Station_ID').sum()
    temp_data[:] = 1
    merge_data = pd.merge(df2,temp_data,how='left',on='Station_ID')

    return merge_data

## 스테이션별 평균 이용객 구하기
 - 이건 학습용 데이터에만 사용되기 때문에 굳이 함수로 생성하지 않는다.

#### 운영 이력 데이터 확인
 - 우린 여기서 스테이션 및 연도별 빌린 횟수만 가져올거기 때문에 불필요한 컬럼 제거

In [ ]:
oper_01

In [ ]:
# 연도와 스테이션별로 groupby해서 각 연도별 스테이션 사용횟수를 구함
# 우리는 일평균이용횟수를 이요할거기 때문에 구한 값
oper_01['year'] = oper_01['LEAS_DATE'].apply(lambda x:x[:4])
oper_01_대여 = oper_01.groupby(['year', 'LEAS_STATION'])['LEAS_NO'].agg(['count']).reset_index()
a = oper_01_대여.copy()
oper_01_대여['count'] = a['count']/365
oper_01_대여.rename(columns={'LEAS_STATION':'Station_ID','count':'일일평균이용수'}, inplace=True)
oper_01_대여

,year,Station_ID,일일평균이용수
0,2017,101,8.254795
1,2017,103,5.079452
2,2017,104,14.810959
3,2017,105,2.657534
4,2017,106,4.441096
...,...,...,...
451,2019,349,6.717808
452,2019,350,3.484932
453,2019,351,1.295890
454,2019,992,1.071233


## 위 함수들 이용한 model 데이터 생성

In [ ]:
# 위 함수들을 이용하여 모델링용 데이터 구하기
model_data = buliding_count(station_2) ##!!!! 이부분이 몇시간 정도 소요됨. 
model_data = pop_count(station_2['경도'], station_2['위도'],model_data)
model_data = bus_count(station_2,model_data)
model_data = sub_count(station_2,model_data)
model_data = park_count(station_2,model_data)
model_data = pd.merge(oper_01_대여,model_data,how='left',on='Station_ID')

#년도 칼럼은 이제 필요 없으므로 제거해준다.
model_data.drop(columns={'year'},inplace=True)

# 2. 일일평균이용수 예측 모델링
- randomforest 모델과 catboost 모델을 이용하였다.

### 데이터 셋 정규화 및 정규화 된 데이터로 모델링
 - 데이터 정규화 작업을 먼저 거쳐준다.

In [ ]:
# 학습시킬 데이터와 예측하고자 하는 데이터로 분할
model_data_x = merge_data.drop(columns={'일일평균이용객'})
model_data_y = merge_data['일일평균이용객']

In [ ]:
# 데이터 정규화 진행
scaler = MinMaxScaler()
model_data_x[:] = scaler.fit_transform(model_data_x[:])
model_data_x.head()

In [ ]:
## 트레이닝셋과 테스트셋 설정
# 트레이닝 셋 : 테스트셋 8:2 비율로 만듬
x_data = model_data_x.drop(columns={'Station_ID'})
y_data = model_data_y

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,random_state=123)

### RandomForest 모델 학습 및 결과

In [ ]:
# grid search를 통해 파라미터 선정 - optimal
rf_model = RandomForestRegressor(n_estimators=1000, max_depth=14)
rf_model.fit(x_train,y_train)

In [ ]:
# 스코어 값 확인
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

### 요인별 중요도 시각화

In [ ]:
def plot_feature_importance(model, X_train, figsize=(12, 6)):
    sns.set_style('darkgrid')
    
    # Plot feature importance
    feature_importance = model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    sorted_idx = sorted_idx[150:]
    pos = np.arange(sorted_idx.shape[0]) + .5

    plt.figure(figsize=figsize)
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X_train.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()

    print(X_train.columns[sorted_idx]) # 그래프에 표현시 글자가 깨져서 이와 같이 따로 출력

In [ ]:
plot_feature_importance(model, x_train)

## catboost 모델 학습 및 결과

In [ ]:
# grid search를 통해 파라미터 선정
cat_model = CatBoostRegressor(n_estimators=3000,learning_rate=0.01,max_depth=11,loss_function='MAE')
cat_model.fit(x_train,y_train)

In [ ]:
# 스코어 값 확인
print(cat_model.score(x_train, y_train))
print(cat_model.score(x_test,y_test))

# 3. 후보 스테이션 위치 및 요인 데이터 생성
 - 먼저 인구분포도 파일을 통해 균일한 좌표값을 생성하고, 생성한 좌표에 대해서 위 데이터 전처리 방식과 동일하게 데이터를 만들어줄 계획
 - 이후 동별 성격 군집화에 따른 조정이나 추가가 필요한 지역만 스테이션 추가 제거 진행
 - 신규 개발지역은 데이터가 부족하여, 현재 어느정도 진행이 된 신사동을 예로 하여 신규개발지역에 스테이션 배치 테스트

 - 추가지역 : 마두2동, 일산3동, 정발산동, 주엽1동, 주엽2동

 - 조정필요지역 : 대화동, 마두1동, 백석1동, 중산동

 - 신규개발지역 : 식사동

## 고양시 전체에 균일하게 좌표 찍기
- 인구 분포가 0인 곳을 제외하고 후보 스테이션을 10,000개 이상 만든다고 가정.
- 인구분포도의 셀 당 스테이션 갯수는 4개

In [ ]:
후보_스테이션_구분 = int(np.ceil((10000 / sum(spread_pop['val'] > 0))**0.5))
후보_스테이션_구분 * 후보_스테이션_구분

4

In [ ]:
minx, miny, maxx, maxy = spread_pop.loc[spread_pop['val'] > 0, 'geometry'].bounds.values.T
dx = (maxx - minx) / 후보_스테이션_구분
dy = (maxy - miny) / 후보_스테이션_구분
xs = minx.reshape([-1, 1, 1]) + (dx.reshape([-1, 1, 1]) * (np.arange(후보_스테이션_구분) + 0.5).reshape([1, -1, 1]))
ys = miny.reshape([-1, 1, 1]) + (dy.reshape([-1, 1, 1]) * (np.arange(후보_스테이션_구분) + 0.5).reshape([1, 1, -1]))
xs = xs + np.zeros_like(ys)
ys = ys + np.zeros_like(xs)
후보_스테이션_경위도 = pd.DataFrame(columns=['경도', '위도'], data={'경도': xs.ravel(), '위도': ys.ravel()})
후보_스테이션_경위도

,경도,위도
0,126.680307,37.683719
1,126.680307,37.684174
2,126.680879,37.683719
3,126.680879,37.684174
4,126.680297,37.684621
...,...,...
22903,126.959892,37.681275
22904,126.969701,37.656533
22905,126.969701,37.656986
22906,126.970271,37.656533


#### 추가해야 하는 지역 : 마두2동, 일산3동, 정발산동, 주엽1동, 주엽2동

In [ ]:
add_dong = dong_point[(dong_point['행정동명'].isin([
                                  '마두2동',
                                  '일산3동',
                                  '정발산동',
                                  '주엽1동',
                                  '주엽2동']))]
add_dong

,행정동코드,행정동명,geometry
21,3110353,정발산동,"MULTIPOLYGON (((126.78686 37.67100, 126.78696 ..."
25,3110357,마두2동,"MULTIPOLYGON (((126.78662 37.65501, 126.78686 ..."
32,3110453,일산3동,"MULTIPOLYGON (((126.76251 37.68582, 126.76283 ..."
34,3110455,주엽1동,"MULTIPOLYGON (((126.76867 37.67494, 126.77080 ..."
35,3110456,주엽2동,"MULTIPOLYGON (((126.76079 37.67829, 126.76105 ..."


#### 조정이 필요한 지역 : 대화동, 마두1동, 백석1동, 중산동

In [ ]:
replace_dong = dong_point[(dong_point['행정동명'].isin([
                                  '대화동',
                                  '마두1동',
                                  '백석1동',
                                  '중산동']))]
replace_dong

,행정동코드,행정동명,geometry
20,3110352,중산동,"MULTIPOLYGON (((126.78349 37.69875, 126.78457 ..."
23,3110355,백석1동,"MULTIPOLYGON (((126.79854 37.65095, 126.79868 ..."
24,3110356,마두1동,"MULTIPOLYGON (((126.78161 37.66891, 126.78324 ..."
36,3110457,대화동,"MULTIPOLYGON (((126.76135 37.68712, 126.76187 ..."


#### 신규 개발지역 : 식사동

In [ ]:
식사동 = dong_point[(dong_point['행정동명'] == '식사동')]
식사동

,행정동코드,행정동명,geometry
19,3110351,식사동,"MULTIPOLYGON (((126.82024 37.69395, 126.82026 ..."


### 스테이션과 동 매칭시키기

####추가 지역 

In [ ]:
count = 0
add_dong_list = []

for i in range(len(후보_스테이션_경위도)):
  for j in range(len(add_dong)):
    if Point(후보_스테이션_경위도.iloc[i].경도, 후보_스테이션_경위도.iloc[i].위도).within(add_dong.iloc[j].geometry) :
       add_dong_list.append([i,add_dong.iloc[j].행정동명])
      

add_dong_df = pd.DataFrame(add_dong_list)
add_dong_df.rename(columns = {0 : 'Station_ID', 1 : '행정동명'}, inplace = True)
후보스테이션_add_dong_df = pd.merge(후보_스테이션_경위도, add_dong_df, left_index = True, right_on = 'Station_ID')
후보스테이션_add_dong_df #데이터 확인

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,경도,위도,Station_ID,행정동명
0,126.751906,37.668873,2488,주엽2동
1,126.751906,37.669327,2489,주엽2동
2,126.752477,37.668873,2490,주엽2동
3,126.752477,37.669327,2491,주엽2동
4,126.751897,37.669774,2492,주엽2동
...,...,...,...,...
1768,126.787634,37.668643,7903,정발산동
1769,126.787054,37.669091,7904,정발산동
1770,126.787054,37.669545,7905,정발산동
1771,126.787625,37.669091,7906,정발산동


#### 조정지역

In [ ]:
# 무작위로 생성한 스테이션 중 원하는 지역 내의 스테이션 선별

count = 0
replace_dong_list = []

for i in range(len(후보_스테이션_경위도)):
  for j in range(len(replace_dong)):
    if Point(후보_스테이션_경위도.iloc[i].경도, 후보_스테이션_경위도.iloc[i].위도).within(replace_dong.iloc[j].geometry) :
       replace_dong_list.append([i,replace_dong.iloc[j].행정동명])
       
replace_dong_df = pd.DataFrame(replace_dong_list)
replace_dong_df.rename(columns = {0 : 'Station_ID', 1 : '행정동명'}, inplace = True)
후보스테이션_replace_dong_df = pd.merge(후보_스테이션_경위도, replace_dong_df, left_index = True, right_on = 'Station_ID')
후보스테이션_replace_dong_df ##데이터 확인

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,경도,위도,Station_ID,행정동명
0,126.736594,37.670127,1607,대화동
1,126.737701,37.672838,1663,대화동
2,126.737691,37.673285,1666,대화동
3,126.737691,37.673739,1667,대화동
4,126.738263,37.672845,1725,대화동
...,...,...,...,...
2107,126.795093,37.658323,9100,마두1동
2108,126.796286,37.652021,9212,백석1동
2109,126.796286,37.652475,9213,백석1동
2110,126.796857,37.652021,9214,백석1동


#### 신규개발지역

In [ ]:
count = 0
식사동_list = []

for i in range(len(후보_스테이션_경위도)):
  for j in range(len(식사동)):
    if Point(후보_스테이션_경위도.iloc[i].경도, 후보_스테이션_경위도.iloc[i].위도).within(식사동.iloc[j].geometry) :
       식사동_list.append([i,식사동.iloc[j].행정동명])

develop_dong_df = pd.DataFrame(식사동_list)
develop_dong_df.rename(columns = {0 : 'Station_ID', 1 : '행정동명'}, inplace = True)
후보스테이션_develop_dong_df = pd.merge(후보_스테이션_경위도, develop_dong_df, left_index = True, right_on = 'Station_ID')
후보스테이션_develop_dong_df #데이터 확인

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,경도,위도,Station_ID,행정동명
0,126.798913,37.675020,9543,식사동
1,126.798904,37.675467,9546,식사동
2,126.798904,37.675921,9547,식사동
3,126.798896,37.676369,9550,식사동
4,126.798896,37.676823,9551,식사동
...,...,...,...,...
723,126.823761,37.685981,12455,식사동
724,126.824397,37.677422,12584,식사동
725,126.824397,37.677876,12585,식사동
726,126.824968,37.677422,12586,식사동


### 1번과 동일하게 좌표 기준 데이터 생성

- 너무 많은 좌표로 한 파일에 모두 돌리게 될 시 2~3일의 시간이 소요되어 데이터를 첨부합니다. (buliding_count 함수가 시간이 오래 걸림)
- 좌표당 일일평균대여수를 구하는 모델을 돌릴 데이터이기 떄문에 방식은 1.전처리와 동일하게 진행되어 건물, 주변인구, 버스, 지하철, 공원 데이터를 구하게 됩니다.
- 혹시 전체 코드는 같이 주석으로 첨부하겠습니다.
- 파일은 알집에 같이 묶어서 첨부하겠습니다.

#### 추가지역

In [ ]:
# model_data = buliding_count(후보스테이션_add_dong_df) ##많은 시간 소요됨 
# model_data = pop_count(후보스테이션_add_dong_df['경도'], 후보스테이션_add_dong_df['위도'],model_data)
# model_data = bus_count(후보스테이션_add_dong_df,model_data)
# model_data = sub_count(후보스테이션_add_dong_df,model_data)
# model_data = park_count(후보스테이션_add_dong_df,model_data)

#### 조정지역

In [ ]:
# model_data = buliding_count(후보스테이션_develop_dong_df) ##많은 시간 소요됨 
# model_data = pop_count(후보스테이션_develop_dong_df['경도'], 후보스테이션_develop_dong_df['위도'],model_data)
# model_data = bus_count(후보스테이션_develop_dong_df,model_data)
# model_data = sub_count(후보스테이션_develop_dong_df,model_data)
# model_data = park_count(후보스테이션_develop_dong_df,model_data)

#### 신규개발지역

In [ ]:
# model_data = buliding_count(후보스테이션_develop_dong_df) ##많은 시간 소요 파트
# model_data = pop_count(후보스테이션_develop_dong_df['경도'], 후보스테이션_develop_dong_df['위도'],model_data)
# model_data = bus_count(후보스테이션_develop_dong_df,model_data)
# model_data = sub_count(후보스테이션_develop_dong_df,model_data)
# model_data = park_count(후보스테이션_develop_dong_df,model_data)

#### 추가지역 데이터 불러오기

In [ ]:
station_대화동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_대화동_데이터.csv")
station_마두1동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_마두1동_데이터.csv")
station_백석1동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_백석1동_데이터.csv")
station_주엽1동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_주엽1동_데이터.csv")
station_주엽2동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_주엽2동_데이터.csv")

#### 조정지역 데이터 불러오기

In [ ]:
station_일산3동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_일산3동_데이터.csv")
station_정발산동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_정발산동_데이터.csv")
station_중산동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_중산동_데이터.csv")
station_마두2동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_마두2동_데이터.csv")

#### 신규개발지역 데이터 불러오기

In [ ]:
station_식사동 = pd.read_csv("/content/drive/My Drive/고양시 공공자전거 위치선정 프로젝트/SBJ_2007_001/스테이션후보군_데이터완성/스테이션후보군_식사동_데이터.csv")

## catboost 모델로 좌표별 일일평균이용수 구하기

### 먼저 요인 동일하게 맞춰주기
 - 건물이 주변에 없는 경우, 아예 column이 생성이 안됐기 때문에 동일하게 column들을 맞춰주기 위해서 데이터들을 정리해줍니다.

In [ ]:
temp_df1 = model_data.copy()
temp_df1.drop(columns={'일일평균이용객'}, inplace=True)
col_list1 = list(temp_df1.columns)
col_list1

NameError: ignored

### 학습된 모델을 이용하여 각 좌표별 예측값 구하기

In [ ]:
# 위 10개 동 좌표 데이터의 columns을 모델 학습데이터와 동일하게 맞춰줍니다.
def for_predict_use(df):
  df = pd.DataFrame(df,columns=a)
  df = df.fillna(0)
  #학습된 catboost 모델을 가지고 예측값을 구하여 일일평균이용수에 넣어줍니다.
  df = pd.merge(df,pd.DataFrame(cat_model.predict(df), columns=['일일평균이용수']) ,left_index=True,right_index=True)
  return df

In [ ]:
station_백석1동 = for_predict_use(station_백석1동)
station_마두1동 = for_predict_use(station_마두1동)
station_마두2동 = for_predict_use(station_마두2동)
station_대화동 = for_predict_use(station_대화동)
station_일산3동 = for_predict_use(station_일산3동)
station_정발산동 = for_predict_use(station_정발산동)
station_주엽1동 = for_predict_use(station_주엽1동)
station_주엽2동 = for_predict_use(station_주엽2동)
station_중산동 = for_predict_use(station_중산동)
station_식사동 = for_predict_use(station_식사동)

#### 위 데이터에 좌표값 넣어주기

In [ ]:
# 간단한 병합 함수 생성
def 병합(df, df2) :
    a = pd.merge(df, df2, on = 'Station_ID', how = 'left')
    return a

In [ ]:
station_대화동 = 병합(station_대화동, 후보스테이션_replace_dong_df)
station_마두1동 = 병합(station_마두1동, 후보스테이션_replace_dong_df)
station_중산동 = 병합(station_중산동, 후보스테이션_replace_dong_df)
station_백석1동 = 병합(station_백석1동, 후보스테이션_replace_dong_df)
station_마두2동 = 병합(station_마두2동, 후보스테이션_add_dong_df)
station_일산3동 = 병합(station_일산3동, 후보스테이션_add_dong_df)
station_주엽1동 = 병합(station_주엽1동, 후보스테이션_add_dong_df)
station_주엽2동 = 병합(station_주엽2동, 후보스테이션_add_dong_df)
station_정발산동 = 병합(station_정발산동, 후보스테이션_add_dong_df)
station_식사동 = 병합(station_식사동, 후보스테이션_develop_dong_df)

# 4. clustering을 통해 후보 스테이션생성

## 각 동별로 clustering을 통한 최종 후보지 선정
 - 추가해야하는 지역과 재조정및 재거가 필요한 지역의 clustering 개수가 다르다.
 - clustering 시 가중치로 각 지점의 예상 수요량을 준다.

#### 조정 지역 clustering

In [ ]:
# 제거 및 재조정이 필요한 지역은 현재 있는 스테이션에서 1개를 제외하여 최종 후보지를 선정
def kmeans_clustering_replace(df1,df2):
  a = df1.iloc[0].행정동명
  b = df1.copy()
  df2 = df2[df2['DONG'] == a]
  kmeans = KMeans(n_clusters=(len(df2)-1)) #cluster 수가 하나 적음
  kmeans.fit(
      X=np.stack([df1['경도'], df1['위도']], axis=1),
      sample_weight=b['일일평균이용수']
  )

  return kmeans

#### 추가지역 clustering

In [ ]:
# 이용량이 많아 추가가 필요한 지역은 cluster 포인트를 기존 스테이션수와 동일하게
def kmeans_clustering_add(df1,df2):
  a = df1.iloc[0].행정동명
  b = df1.copy()
  df2 = df2[df2['DONG'] == a]
  kmeans = KMeans(n_clusters=len(df2)) # cluster수가 동일하다
  kmeans.fit(
      X=np.stack([df1['경도'], df1['위도']], axis=1),
      sample_weight=b['일일평균이용수']
  )

  return kmeans

### 현재 기존 스테이션 만들기

In [ ]:
station_dong = []

for i in range(len(station_2)):
  for j in range(len(dong_point)):
    if Point(station_2.iloc[i].경도,station_2.iloc[i].위도).within(dong_point.iloc[j].geometry) :
       station_dong.append([dong_point.iloc[j].행정동명,station_2.iloc[i].Station_ID])
       break;

station_dong = pd.DataFrame(station_dong)
station_dong.rename(columns={0:'DONG',1:'Station_ID'},inplace=True)
#oper_01_대여에서 최신 2019년 값만 가져오기
origin_station = pd.merge(oper_01_대여[oper_01_대여['year'] == '2019'],station_dong,how='left',on='Station_ID')
origin_station = origin_station.fillna(0)
origin_station.drop(columns={'year'},inplace=True)
temp = station_2[['Station_ID','거치대 수량','위도','경도']]

origin_station = pd.merge(origin_station,temp,how='left',on='Station_ID')
origin_station = origin_station[origin_station['DONG'] != 0]
origin_station

,Station_ID,일일평균이용수,DONG,거치대 수량,위도,경도
0,101,7.769863,성사2동,20.0,37.654775,126.834584
1,103,3.980822,성사1동,20.0,37.660442,126.840377
2,104,10.389041,탄현동,25.0,37.698523,126.766042
3,105,2.868493,성사1동,20.0,37.655244,126.839261
4,106,4.356164,성사1동,30.0,37.653410,126.842530
...,...,...,...,...,...,...
150,348,5.082192,송산동,30.0,37.699353,126.754793
151,349,6.717808,송산동,20.0,37.697838,126.752642
152,350,3.484932,송산동,20.0,37.697867,126.753089
153,351,1.295890,탄현동,40.0,37.702259,126.767231


#### 추가지역 클러스터 결과

In [ ]:
kmeans_일산3동 = kmeans_clustering_add(station_일산3동,origin_station)
kmeans_마두2동 = kmeans_clustering_add(station_마두2동,origin_station)
kmeans_주엽1동 = kmeans_clustering_add(station_주엽1동,origin_station)
kmeans_주엽2동 = kmeans_clustering_add(station_주엽2동,origin_station)
kmeans_정발산동 = kmeans_clustering_add(station_정발산동,origin_station)

#### 조정지역 클러스터 결과

In [ ]:
kmeans_대화동 = kmeans_clustering_replace(station_대화동,origin_station)
kmeans_마두1동 = kmeans_clustering_replace(station_마두1동,origin_station)
kmeans_중산동 = kmeans_clustering_replace(station_중산동,origin_station)
kmeans_백석1동 = kmeans_clustering_replace(station_백석1동,origin_station)


## 최종 후보지 사용 예측값 구하기
 - clustering 후 나온 최종 후보지에서 가장 근접한 point의 일일사용예측치를 최종 후보지의 평균이용수로 사용한다.

In [ ]:
def station_use(df1,df2):
  temp_list = []
  for i in range(len(df1.cluster_centers_)):
    temp = 100000
    use = 0
    for j in range(len(df2)):
       x = (df1.cluster_centers_[i][0],df1.cluster_centers_[i][1])
       y = (df2.iloc[j].경도,df2.iloc[j].위도)
       result1 = haversine(x,y,unit='m')
       if result1 < temp :
          temp = result1
          use = df2.iloc[j].일일평균이용수
       else :
          pass
    temp_list.append([df1.cluster_centers_[i][0],df1.cluster_centers_[i][1],use,df2.iloc[0].행정동명])
  temp_list = pd.DataFrame(temp_list)
  temp_list.rename(columns={0:'경도',1:'위도',2:'일일평균이용수',3:'행정동명'},inplace=True)
  return temp_list

In [ ]:
kmeans_point_대화동 = station_use(kmeans_대화동,스테이션후보_대화동)
kmeans_point_마두1동 = station_use(kmeans_마두1동,스테이션후보_마두1동)
kmeans_point_마두2동 = station_use(kmeans_마두2동,스테이션후보_마두2동)
kmeans_point_중산동 = station_use(kmeans_중산동,스테이션후보_중산동)
kmeans_point_백석1동 = station_use(kmeans_백석1동,스테이션후보_백석1동)
kmeans_point_일산3동 = station_use(kmeans_일산3동,스테이션후보_일산3동)
kmeans_point_주엽1동 = station_use(kmeans_주엽1동,스테이션후보_주엽1동)
kmeans_point_주엽2동 = station_use(kmeans_주엽2동,스테이션후보_주엽2동)
kmeans_point_정발산동 = station_use(kmeans_정발산동,스테이션후보_정발산동)

# 5. 현 스테이션과 후보 스테이션과의 비교를 통한 스테이션 추가 및 조정

- 스테이션 조정 및 추가 설치 목적은 접근성 증대 및 효율성 증대에 목적이 있음.
- 추가 설치 지역은 다른 동에 비해 월등하게 이용횟수가 많지만 스테이션 개수는 그에 비해 적으므로 적어도 한개 이상의 스테이션을 신설할 계획
 
- 추가만 하는 지역 
  - 1 :하나의 신규 스테이션 후보지와 기존 스테이션과의 거리 비교 / 가장 가까운 거리 저장(이 작업을 모든 신규스테이션에 반복) -> 저장된 가까운 거리 중 가장 max값인 곳에 스테이션을 설치
    - 위 같이 설치하는 이유는 현재 설치하려는 지역에서 기존 스테이션이 커버하지 못하는 지역에 신설 스테이션을 설치하여 접근성을 높이기 위함(기존 스테이션과 가장 멀리 떨어진 신규지역에 새로운 스테이션 설치)
  - 2: 이용률이 가장 높은 지역에 근접한 곳에 추가 스테이션 설치로 이용률 분산
- 조정 및 추가하는 지역
  - 1 최하위 제거 : 하나 이상의 스테이션 무조건 제거 
  - 2 추가 : 기존 스테이션 평균보다 이용횟수가 높은 신규 스테이션 추가 설치 (단, 같은 블록 약 300m 내는 기존 스테이션이 있을 경우 설치 불가)

- 삭제 추가 여부 칼럼은 
  - 0: 기존 스테이션
  - 1: 삭제 스테이션
  - 2: 신규 스테이션

## 추가해야 하는 지역 

In [ ]:
# 이 지역은 접근성을 높여야 한다.
# 스테이션 추가설치가 필요한 지역 함수
# df1 : 기존 스테이션, df2 : 신규 스테이션 후보
# 리턴값은 해당 동의 신규 + 현재 스테이션
def add_algo(df1, df2):
  add_list = []
  a = df2.iloc[0].행정동명
  df1 = df1[df1['DONG'] == a] # 기존 스테이션에서 필요 동만 추출
  max_dis = 0 # max 거리 저장할 변수
  point_index = 100 # 설치할 지역 인덱스 저장

  # 1번 추가 알고리즘
  for i in range(len(df1)):
    temp = 1000000 # 임의의 큰 값 부여
    z = 0 # 포인트 저장
    for j in range(len(df2)):
       x = (df1.iloc[i].경도,df1.iloc[i].위도)
       y = (df2.iloc[j].경도,df2.iloc[j].위도)
       result1 = haversine(x,y,unit='m')
       if result1 < temp :
          temp = result1
          z = j
       else :
          pass
    if max_dis < temp: # 최대 거리 찾기
      max_dis = temp
      point_index = z
  add_list.append([df2.iloc[point_index].경도,df2.iloc[point_index].위도, df2.iloc[point_index].일일평균이용수])


  # 2번 추가 알고리즘
  max_use = df1.일일평균이용수.max() # 가장 이용율 높은 스테이션에 근접한 곳에 추가설치
  high_station = df1[df1['일일평균이용수'] == max_use]
  for i in range(len(df2)):
    x = (high_station.iloc[0].경도,high_station.iloc[0].위도)
    y = (df2.iloc[i].경도,df2.iloc[i].위도)
    result1 = haversine(x,y,unit='m')
    temp = 100000
    if result1 < temp:
      temp = result1
      point_index2 = i
  if point_index2 != point_index :
    add_list.append([df2.iloc[point_index2].경도,df2.iloc[point_index2].위도, df2.iloc[point_index2].일일평균이용수])
  else :
    pass

  df1['삭제추가여부'] = 0
  add_list = pd.DataFrame(add_list)
  add_list.rename(columns={0:'경도',1:'위도',2:'일일평균이용수'}, inplace=True)
  add_list['거치대 수량'] = np.where(add_list['일일평균이용수']<10,
                                15,(np.where(add_list['일일평균이용수']<20,20,
                                                                    np.where(add_list['일일평균이용수']<40,25,30))))
  add_list['DONG'] = a
  add_list['삭제추가여부'] = 2
  col_list = ['Station_ID','거치대 수량','위도','경도','DONG','일일평균이용수','삭제추가여부']
  add_list = pd.DataFrame(add_list,columns=col_list)
  result_df = pd.concat([df1,add_list])
  result_df.reset_index(drop =True, inplace=True)
                        
  return result_df

In [ ]:
# 추가해야할 지역 -> 마두2동/일산3동/정발산동/주엽1동/주엽2동
point_마두2동 = add_algo(origin_station,kmeans_point_마두2동)
point_일산3동 = add_algo(origin_station,kmeans_point_일산3동)
point_정발산동 = add_algo(origin_station,kmeans_point_정발산동)
point_주엽1동 = add_algo(origin_station,kmeans_point_주엽1동)
point_주엽2동 = add_algo(origin_station,kmeans_point_주엽2동)

## 조정이 필요한 지역

In [ ]:
# 여기서는 효율성을 챙겨야 한다.
# 스테이션 조정 필요한 지역 함수
# df1 : 기존 스테이션, df2 : 신규 스테이션 후보
# 리턴값은 해당 동의 신규 + 현재 스테이션 중 남아있는 스테이션

def replace_algo(df1,df2):
  replace_list = []
  remove_list = []
  a = df2.iloc[0].행정동명
  df1 = df1[df1['DONG'] == a] # 기존 스테이션에서 필요 동만 추출
  df1.sort_values(by='일일평균이용수', ascending=True, inplace=True)
  # 최하위 스테이션 2개의 근방 스테이션 구하는 for문
  for i in range(2):
    temp1 = 100000
    point_index = 0
    for j in range(len(df1)):
      x = (df1.iloc[i].경도,df1.iloc[i].위도)
      y = (df1.iloc[j].경도,df1.iloc[j].위도)
      result1 = haversine(x,y,unit='m')
      if result1 != 0 and result1 < temp1 :
        temp1 = result1
        point_index = j
      else :
        pass
    #최하위스테이션의 근방 스테이션과 그 중점의 가장 가까운 신규 스테이션의 이용횟수 비교
    temp2 = 100000
    new_index = 0
    for k in range(len(df2)): 
      x1 = (((df1.iloc[i].경도)+(df1.iloc[point_index].경도))/2,((df1.iloc[i].위도)+(df1.iloc[point_index].위도))/2)
      y1 = (df2.iloc[k].경도,df2.iloc[k].위도)
      result2 = haversine(x1,y1,unit='m')
      if result2 < temp2 :
        temp2 = result2
        new_index = k
    if df2.iloc[new_index].일일평균이용수 > df1.iloc[point_index].일일평균이용수 :
      replace_list.append([df2.iloc[new_index].경도,df2.iloc[new_index].위도,df2.iloc[new_index].일일평균이용수])
      remove_list.append(point_index)
    else :
      pass
  
  df1.reset_index(drop=True,inplace=True)
  df1['삭제추가여부'] = 0
  df1['삭제추가여부'][0] = 1
  df1['삭제추가여부'][1] = 1
  for i in range(len(remove_list)):
     df1['삭제추가여부'][remove_list[i]] = 1
  if len(replace_list) > 0 :
    replace_list = pd.DataFrame(replace_list)
    replace_list.rename(columns={0:'경도',1:'위도',2:'일일평균이용수'}, inplace=True)
    replace_list['거치대 수량'] = np.where(replace_list['일일평균이용수']<10,
                                15,(np.where(replace_list['일일평균이용수']<20,20,
                                                                    np.where(replace_list['일일평균이용수']<40,25,30))))
    replace_list['DONG'] = a
    replace_list['삭제추가여부'] = 2
    col_list = ['Station_ID','거치대 수량','위도','경도','DONG','일일평균이용수','삭제추가여부']
    replace_list = pd.DataFrame(replace_list,columns=col_list)
    if len(replace_list) == 2 :
       if replace_list.iloc[0].일일평균이용수 == replace_list.iloc[1].일일평균이용수 :
           replace_list = replace_list[1:]
    result_df = pd.concat([df1,replace_list])
    result_df.reset_index(drop =True, inplace=True)
  else :
    result_df = df1
    result_df.reset_index(drop =True, inplace=True)
  return result_df

In [ ]:
# 재배치 및 조정 필요 지역 -> 대화동, 마두1동, 백석1동, 중산동
# 최종 선정 지역!!
point_대화동 = replace_algo(origin_station,kmeans_point_대화동)
point_마두1동 = replace_algo(origin_station,kmeans_point_마두1동)
point_백석1동 = replace_algo(origin_station,kmeans_point_백석1동)
point_중산동 = replace_algo(origin_station,kmeans_point_중산동)

### 신규 개발 지역 따로 클러스터링

In [ ]:
b = station_식사동.copy()
kmeans = KMeans(n_clusters=5) # 해당 동의 일일대여수에 기반한 클러스터 수 
kmeans.fit(
      X=np.stack([station_식사동['경도'], station_식사동['위도']], axis=1),
      sample_weight=b['일일평균이용수']
  )
kmeans_식사동 = kmeans

In [ ]:
develop_station_final = station_use(kmeans_식사동,station_식사동)
develop_station_final['거치대 수량'] = np.where(develop_station_final['일일평균이용수']<10,
                                15,(np.where(develop_station_final['일일평균이용수']<20,20,
                                                                    np.where(develop_station_final['일일평균이용수']<40,25,30))))

# 6. 최종 선정된 후보 스테이션 위치 미세 조정
 - 신규 설치된 스테이션들을 도로 근방으로 조정

### 도로 데이터 확인

In [ ]:
도로명주소_도로

### 곡선도로 직선을 분리

In [ ]:
# 곡선도로 직선으로 분리

x0, x1 = list(), list()
for lines in 도로명주소_도로['geometry']:
    for line in lines:
        for _x0, _y0, _x1, _y1 in zip(line.xy[0][:-1], line.xy[1][:-1], line.xy[0][1:], line.xy[1][1:]):
            x0.append([_x0, _y0])
            x1.append([_x1, _y1])

lines = shapely.ops.cascaded_union([shapely.geometry.LineString([x0, x1]) for x0, x1 in zip(x0, x1)])


m = folium.Map(location = [
        spread_pop['geometry'].apply(lambda x: x.centroid).y.mean(),
        spread_pop['geometry'].apply(lambda x: x.centroid).x.mean()],
        zoom_start = 12,
        # tiles = 'http://api.vworld.kr/req/wmts/1.0.0/4CBE11A5-5002-3452-B407-8F2974422E24/Base/{z}/{y}/{x}.png',
        # attr = '고양시' # 원래 이렇게 맵 표시하는게 맞지만, 현재 오류때문에 그냥 Open street로 표시
              )

folium.Choropleth(
    geo_data=lines,
    line_color='blue',
).add_to(m)


m

### 추가, 조정 스테이션 데이터 확인

In [ ]:
# 아래 나오는 값들이 추가 또는 조정 된 스테이션들
# 대화동, 마두1동, 백석1동, 중산동
# 마두2동/일산3동/정발산동/주엽1동/주엽2동

# 스테이션 추가 및 조정 한 데이터 확인
add_dong_station = pd.concat([point_마두2동, point_일산3동, point_정발산동,
                               point_주엽1동, point_주엽2동])
replace_dong_station = pd.concat([point_대화동, point_마두1동, point_백석1동, point_중산동])


# 군집별 신규 추가된 지역만 뽑아내기
add_dong_station_new = add_dong_station[add_dong_station['삭제추가여부'] == 2].reset_index(drop=True)

replace_dong_station_new = replace_dong_station[replace_dong_station['삭제추가여부'] == 2].reset_index(drop=True)


### 직선 내 가장 가까운 점 찾기

In [ ]:
def find_nearest_point(x0, x1, x2, a):
    x0 = tf.reshape(tf.constant(x0, dtype=tf.float32), [-1, 2, 1])
    x1 = tf.reshape(tf.constant(x1, dtype=tf.float32), [-1, 2, 1])
    x2 = tf.reshape(tf.constant(x2, dtype=tf.float32), [-1, 2, 1])
    A = tf.reshape(
        tf.stack([
            (x0[:, 1, 0] - x1[:, 1, 0]), -(x0[:, 0, 0] - x1[:, 0, 0]),
            (x0[:, 0, 0] - x1[:, 0, 0]),  (x0[:, 1, 0] - x1[:, 1, 0]),
        ], axis=1),
        [-1, 2, 2],
    )
    A_I = tf.reshape(
        tf.stack([
             (x0[:, 1, 0] - x1[:, 1, 0]), (x0[:, 0, 0] - x1[:, 0, 0]),
            -(x0[:, 0, 0] - x1[:, 0, 0]), (x0[:, 1, 0] - x1[:, 1, 0]),
        ], axis=1)  / tf.reshape((x0[:, 0, 0] - x1[:, 0, 0])**2 + (x0[:, 1, 0] - x1[:, 1, 0])**2, [-1, 1]),
        [-1, 2, 2],
    )
    c_0 = tf.matmul(A, x0)
    c_1 = tf.matmul(A, x1)
    c_2 = tf.matmul(A, x2)
    c_01 = tf.stack([c_0[:, 1, 0], c_1[:, 1, 0]], axis=1)
    c_min = tf.reduce_min(c_01, axis=1)
    c_max = tf.reduce_max(c_01, axis=1)
    c = tf.reduce_max(
        tf.stack([
            tf.reduce_min(tf.stack([c_2[:, 1, 0], c_max], axis=1), axis=1),
            c_min,
        ], axis=1),
        axis=1,
    )
    y = tf.reshape(
        tf.stack([c_0[:, 0, 0], c], axis=1),
        [-1, 2, 1],
    )
    x4 = tf.matmul(A_I, y)
    dist = tf.reduce_sum((tf.reshape(x2, [-1, 2]) - tf.reshape(x4, [-1, 2]))**2, axis=1)
    idx = tf.argmin(dist)
    res = x4[idx, :, 0]
    return res[0].numpy(), res[1].numpy(), a

In [ ]:
# 함수를 적용하여 직선과 가까운 위치로 스테이션 재조정
temp_add = []
temp_replace = []
temp_develop = []
for i in range(len(add_dong_station_new)):
    temp_add.append(find_nearest_point(x0, x1, (add_dong_station_new.iloc[i].경도,add_dong_station_new.iloc[i].위도), add_dong_station_new.iloc[i].일일평균이용수))
for i in range(len(replace_dong_station_new)):
    temp_replace.append(find_nearest_point(x0, x1, (replace_dong_station_new.iloc[i].경도,replace_dong_station_new.iloc[i].위도), replace_dong_station_new.iloc[i].일일평균이용수))
for i in range(len(develop_station_final)):
    temp_develop.append(find_nearest_point(x0, x1, (develop_station_final.iloc[i].경도,develop_station_final.iloc[i].위도), develop_station_final.iloc[i].일일평균이용수))

## 지역별 최종 결과
- 설치가 가능하도록 인근 도로변으로 스테이션의 위치를 조정

In [ ]:
# 임시 list 데이터프레임으로 변경 후 기존 데이터와 merge

temp_add = pd.DataFrame(temp_add)
temp_add.rename(columns = {0 : '경도', 1: '위도', 2: '일일평균이용수'}, inplace = True)
add_dong_station_new.drop(columns={'경도','위도'},inplace=True)

add_dong_station_new = pd.merge(temp_add,add_dong_station_new,how='left',on='일일평균이용수')
print(add_dong_station_new)


temp_replace = pd.DataFrame(temp_replace)
temp_replace.rename(columns = {0 : '경도', 1: '위도', 2: '일일평균이용수'}, inplace = True)
replace_dong_station_new.drop(columns={'경도','위도'},inplace=True)

replace_dong_station_new = pd.merge(temp_replace,replace_dong_station_new,how='left',on='일일평균이용수')
print(replace_dong_station_new)

temp_develop = pd.DataFrame(temp_develop)
temp_develop.rename(columns = {0 : '경도', 1: '위도', 2: '일일평균이용수'}, inplace = True)
develop_station_final_new.drop(columns={'경도','위도'},inplace=True)

develop_station_final = pd.merge(temp_develop,develop_station_final_new,how='left',on='일일평균이용수')
print(develop_station_final)

In [ ]:
add_dong_station_final = pd.concat([add_dong_station[add_dong_station['삭제추가여부'] != 2],add_dong_station_new])
add_dong_station_final.reset_index(drop = True, inplace=True)

replace_dong_station_final = pd.concat([replace_dong_station[replace_dong_station['삭제추가여부'] != 2],replace_dong_station_new]) 
replace_dong_station_final.reset_index(drop = True, inplace=True)   

# 7.시각화

## 추가 지역

In [ ]:
# 추가 지역 확인
add_dong_station_final

In [ ]:
m = folium.Map(location = [
        인구분포도['geometry'].apply(lambda x: x.centroid).y.mean(),
        인구분포도['geometry'].apply(lambda x: x.centroid).x.mean()],
        zoom_start = 12,
        tiles = 'http://api.vworld.kr/req/wmts/1.0.0/4CBE11A5-5002-3452-B407-8F2974422E24/Base/{z}/{y}/{x}.png',
        attr = '고양시' # 원래 이렇게 맵 표시하는게 맞지만, 현재 오류때문에 그냥 Open street로 표시
              )

folium.GeoJson(dong_point).add_to(m)

# 검은색은 기존 스테이션
for index, row in add_dong_station_final[add_dong_station_final['삭제추가여부'] == 0].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='black', fill_color='black',
                            popup = (str(row['일일평균이용수']))).add_to(m)

# 빨간색은 추가 스테이션
for index, row in add_dong_station_final[add_dong_station_final['삭제추가여부'] == 2].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m)   
m

## 조정지역

In [ ]:
# 조정 지역 확인
replace_dong_station_final

In [ ]:
m = folium.Map(location = [
        인구분포도['geometry'].apply(lambda x: x.centroid).y.mean(),
        인구분포도['geometry'].apply(lambda x: x.centroid).x.mean()],
        zoom_start = 12,
        tiles = 'http://api.vworld.kr/req/wmts/1.0.0/4CBE11A5-5002-3452-B407-8F2974422E24/Base/{z}/{y}/{x}.png',
        attr = '고양시' # 원래 이렇게 맵 표시하는게 맞지만, 현재 오류때문에 그냥 Open street로 표시
              )

folium.GeoJson(dong_point).add_to(m)

# 검은색은 기존 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 0].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='black', fill_color='black',
                            popup = (str(row['일일평균이용수']))).add_to(m)
# 파란색은 삭제 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 1].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='blue', fill_color= 'blue',
                            popup = (str(row['일일평균이용수']))).add_to(m)

# 빨간색은 추가 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 2].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m)  

  


m

## 신규개발지역

In [ ]:
m = folium.Map(location = [
        인구분포도['geometry'].apply(lambda x: x.centroid).y.mean(),
        인구분포도['geometry'].apply(lambda x: x.centroid).x.mean()],
        zoom_start = 12,
        tiles = 'http://api.vworld.kr/req/wmts/1.0.0/4CBE11A5-5002-3452-B407-8F2974422E24/Base/{z}/{y}/{x}.png',
        attr = '고양시' # 원래 이렇게 맵 표시하는게 맞지만, 현재 오류때문에 그냥 Open street로 표시
              )

folium.GeoJson(dong_point).add_to(m)

# 빨간색은 추가 스테이션
for index, row in develop_station_final_new.iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = (row['일일평균이용수'])/5,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m)  


m

## 전체 지역 시각화

In [ ]:
m = folium.Map(location = [
        인구분포도['geometry'].apply(lambda x: x.centroid).y.mean(),
        인구분포도['geometry'].apply(lambda x: x.centroid).x.mean()],
        zoom_start = 12,
        tiles = 'http://api.vworld.kr/req/wmts/1.0.0/4CBE11A5-5002-3452-B407-8F2974422E24/Base/{z}/{y}/{x}.png',
        attr = '고양시' # 원래 이렇게 맵 표시하는게 맞지만, 현재 오류때문에 그냥 Open street로 표시
              )

# 기존 모든 스테이션 검은색 표시
for index, row in  ori.iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='black', fill_color= 'black',
                            ).add_to(m)  


## 추가지역 데이터---------------------------------------------------------------------------------------
# 검은색은 기존 스테이션
for index, row in add_dong_station_final[add_dong_station_final['삭제추가여부'] == 0].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='black', fill_color='black',
                            popup = (str(row['일일평균이용수']))).add_to(m)

# 빨간색은 추가 스테이션
for index, row in add_dong_station_final[add_dong_station_final['삭제추가여부'] == 2].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m) 

# 조정지역 데이터----------------------------------------------------------------------------------------
# 검은색은 기존 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 0].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='black', fill_color='black',
                            popup = (str(row['일일평균이용수']))).add_to(m)
# 파란색은 삭제 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 1].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='blue', fill_color= 'blue',
                            popup = (str(row['일일평균이용수']))).add_to(m)

# 빨간색은 추가 스테이션
for index, row in replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 2].iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m) 

## 신규개발지역 데이터 ----------------------------------------------------------------------------------
# 빨간색은 추가 스테이션
for index, row in develop_station_final_new.iterrows():

    folium.CircleMarker([row['위도'], row['경도']],
                            radius = 1,
                            color='red', fill_color= 'red',
                            popup = (str(row['일일평균이용수']))).add_to(m)  


  

m

# 8. 최종 스테이션 결과
- 데이터프레임으로 출력

In [ ]:
total_station_result = station_2.copy()
total_station_result.drop(columns={'STATION_NAME'}, inplace=True)
# 추가지역의 삭제와 추가 스테이션 따로 저장
add_dong_station_del = add_dong_station_final[add_dong_station_final['삭제추가여부'] == 1]
add_dong_station_del = add_dong_station_del[['Station_ID','거치대 수량','위도','경도']]
add_dong_station_add = add_dong_station_final[add_dong_station_final['삭제추가여부'] == 2]
add_dong_station_add = add_dong_station_add[['Station_ID','거치대 수량','위도','경도']]
# 조정지역의 삭제와 추가 스테이션 따로 저장
replace_dong_station_del = replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 1]
replace_dong_station_del = replace_dong_station_del[['Station_ID','거치대 수량','위도','경도']]
replace_dong_station_add = replace_dong_station_final[replace_dong_station_final['삭제추가여부'] == 2]
replace_dong_station_add = replace_dong_station_add[['Station_ID','거치대 수량','위도','경도']]

develop_station_final_new = develop_station_final_new[['거치대 수량','위도','경도']]
develop_station_final_new['Station_ID'] = np.nan
for i in list(add_dong_station_del['Station_ID']):
  total_station_result = total_station_result[total_station_result['Station_ID'] != i]
for i in list(replace_dong_station_del['Station_ID']):
  total_station_result = total_station_result[total_station_result['Station_ID'] != i]

total_station_result = pd.concat([total_station_result,add_dong_station_add])
total_station_result = pd.concat([total_station_result,replace_dong_station_add])
total_station_result = pd.concat([total_station_result,develop_station_final_new])
total_station_result.reset_index(drop=True, inplace=True)
total_station_result


[101,
 103,
 104,
 105,
 106,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 146,
 148,
 151,
 152,
 153,
 154,
 155,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 176,
 177,
 178,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 326,
 327,
 328,
 329,
 330,
 331,
 333,
 334,
 339,
 340,
 341,
 342,
 343,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 992]